In [26]:
import openai
import pandas as pd 
import numpy as np
import os
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold


In [ ]:
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

try:
    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[{"role": "user", "content": "Diga apenas: funcionando"}],
        max_tokens=5
    )
    print("✔ A chave está funcionando!")
    print("Resposta:", response.choices[0].message.content)
except Exception as e:
    print("✘ Erro! A chave NÃO está funcionando.")
    print("Detalhes:", e)


✔ A chave está funcionando!
Resposta: funcionando


In [10]:
email = pd.read_csv('./email.csv')
email.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
sample_email = email.sample(100)

In [19]:
lista_respostas = []
sample_email = email.sample(100)

for message in sample_email['Message']:
    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[{"role": "user", "content": "Você é um especialista em detecção de spam. Analise a seguinte mensagem e diga se é spam ou não, sendd 1 para spam e 0 para não. Não responsa nada mais além disso: " + message}],
        max_tokens=5
    )

    resposta = response.choices[0].message.content.strip()
    lista_respostas.append(resposta)



In [ ]:
sample_email['Category_GPT4O'] = lista_respostas

,Category,Message,Category_GPT4O
1485,ham,"Sorry, I'll call later",0
3481,ham,What was she looking for?,0
1255,ham,What your plan for pongal?,0
374,ham,I cant keep talking to people if am not sure i...,0
3575,ham,Yeah sure I'll leave in a min,0


In [23]:
sample_email['Category_int'] = sample_email['Category'].apply(lambda x: 1 if x == 'spam' else 0)

In [25]:
# metrics 

precision = precision_score(sample_email['Category_int'], sample_email['Category_GPT4O'].astype(int))
recall = recall_score(sample_email['Category_int'], sample_email['Category_GPT4O'].astype(int))
f1 = f1_score(sample_email['Category_int'], sample_email['Category_GPT4O'].astype(int))
accuracy = accuracy_score(sample_email['Category_int'], sample_email['Category_GPT4O'].astype(int))

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Accuracy: {accuracy:.2f}")

Precision: 0.80
Recall: 1.00
F1 Score: 0.89
Accuracy: 0.96


In [27]:
# ============================================================
# 0. IMPORTS
# ============================================================
from openai import OpenAI
import pandas as pd
import numpy as np
import os

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.model_selection import StratifiedKFold



# ============================================================
# 2. FUNÇÃO: CLASSIFICAR UMA ÚNICA MENSAGEM COM GPT
# ============================================================

def classify_with_gpt(message: str, model_name: str = "gpt-4o-2024-08-06") -> str:
    """
    Usa o GPT para classificar uma mensagem de e-mail como 'spam' ou 'ham'.
    Retorna SEMPRE 'spam' ou 'ham' (em minúsculo).
    """
    if not isinstance(message, str):
        message = str(message)

    prompt = f"""
Você é um especialista em detecção de spam.

Sua tarefa é classificar a mensagem abaixo como:
- "spam" se for uma mensagem indesejada, promocional, fraude, sorteio, etc.
- "ham" se for uma mensagem legítima, pessoal, profissional ou neutra.

Regras IMPORTANTES:
- Responda APENAS com UMA palavra: spam ou ham.
- Não escreva explicações.
- Não traduza.
- Não use frases adicionais.

Mensagem:
\"\"\"{message}\"\"\"
"""

    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=3,
        temperature=0,
    )

    resp = response.choices[0].message.content.strip().lower()

    # Normalização da resposta
    if "spam" in resp and "ham" not in resp:
        return "spam"
    if "ham" in resp and "spam" not in resp:
        return "ham"

    # fallback: se ele inventar moda, consideramos ham (conservador)
    return "ham"


# ============================================================
# 3. CARREGAR O DATASET
# ============================================================

# Ajuste o caminho se necessário
email = pd.read_csv("./email.csv")

# Normaliza colunas
email["Category"] = email["Category"].astype(str).str.lower().str.strip()
email["Message"] = email["Message"].astype(str)

print("Primeiras linhas do dataset:")
print(email.head())

# OPCIONAL: usar uma amostra para reduzir custo e tempo
# (descomente se quiser)
# email = email.sample(200, random_state=42).reset_index(drop=True)

X = email["Message"].reset_index(drop=True)
y = email["Category"].reset_index(drop=True)

print("\nTamanho do conjunto usado:")
print(len(X))


# ============================================================
# 4. K-FOLD USANDO APENAS GPT COMO CLASSIFICADOR
# ============================================================

k = 5  # número de folds

 = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# para guardar todas as previsões (mesmo índice de X/y)
all_preds = pd.Series(index=X.index, dtype=object)

accs, precs, recs, f1s = [], [], [], []

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n===== FOLD {fold}/{k} =====")

    X_test = X.iloc[test_idx]
    y_test = y.iloc[test_idx]

    fold_preds = []

    for msg in X_test:
        pred = classify_with_gpt(msg)   # <<< SOMENTE GPT AQUI
        fold_preds.append(pred)

    fold_preds = np.array(fold_preds)
    all_preds.iloc[test_idx] = fold_preds

    # converte para binário (spam = 1, ham = 0) para métricas numéricas
    y_true_bin = (y_test == "spam").astype(int)
    y_pred_bin = (fold_preds == "spam").astype(int)

    acc = accuracy_score(y_true_bin, y_pred_bin)
    prec = precision_score(y_true_bin, y_pred_bin, zero_division=0)
    rec = recall_score(y_true_bin, y_pred_bin, zero_division=0)
    f1 = f1_score(y_true_bin, y_pred_bin, zero_division=0)

    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1s.append(f1)

    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-score:  {f1:.4f}")

# MÉDIAS FINAIS
print("\n===== MÉDIAS FINAIS (GPT com K-Fold) =====")
print(f"Accuracy médio:  {np.mean(accs):.4f}")
print(f"Precision média: {np.mean(precs):.4f}")
print(f"Recall médio:    {np.mean(recs):.4f}")
print(f"F1-score médio:  {np.mean(f1s):.4f}")


# ============================================================
# 5. OPCIONAL: SALVAR RESULTADOS (RÓTULO REAL + RÓTULO GPT)
# ============================================================

email_result = email.copy().reset_index(drop=True)
email_result["Category_GPT"] = all_preds.values

print("\nPrimeiras linhas com previsão do GPT:")
print(email_result.head())



Primeiras linhas do dataset:
  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...

Tamanho do conjunto usado:
5573

===== FOLD 1/5 =====


c:\Users\xd-bo\Desktop\projects\-UFJF-MS-Computer-Science---Machine-Learning-Work-Spam-Detection-\venv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Accuracy:  0.9534
Precision: 0.7526
Recall:    0.9733
F1-score:  0.8488

===== FOLD 2/5 =====
Accuracy:  0.9480
Precision: 0.7255
Recall:    0.9867
F1-score:  0.8362

===== FOLD 3/5 =====
Accuracy:  0.9498
Precision: 0.7313
Recall:    0.9866
F1-score:  0.8400

===== FOLD 4/5 =====
Accuracy:  0.9479
Precision: 0.7310
Recall:    0.9664
F1-score:  0.8324

===== FOLD 5/5 =====
Accuracy:  0.9470
Precision: 0.7228
Recall:    0.9799
F1-score:  0.8319

===== MÉDIAS FINAIS (GPT com K-Fold) =====
Accuracy médio:  0.9492
Precision média: 0.7326
Recall médio:    0.9786
F1-score médio:  0.8379

Primeiras linhas com previsão do GPT:
  Category                                            Message Category_GPT
0      ham  Go until jurong point, crazy.. Available only ...          ham
1      ham                      Ok lar... Joking wif u oni...          ham
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...         spam
3      ham  U dun say so early hor... U c already then say...          ha